In [1]:
import sys
import zipfile
import os

import numpy as np
import pandas as pd
from IPython.display import display, HTML

sys.path.append("../../libs/")

from datasetutil import gerar_estatiscas_df, substituir_valores_nulos , verify_gg4cc , save_zip

In [2]:

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Metodos

In [3]:
def read(zinput,nome_arquivo,
         file_format=".csv",
         separador=',',codificacao="iso-8859-1"):
    '''
        Ler zip com os datasets
    '''
    zf = zipfile.ZipFile(zinput) 

    for file in zipfile.ZipFile.namelist(zf):
        if file == nome_arquivo+file_format:
            df = pd.read_csv(zf.open(file), header=0, encoding=codificacao,
                             sep=separador,error_bad_lines=False)#,
#             chunksize=640000000,
#             nrows=15)
            return df
# df.iloc[:, [68,2,10,11,9,28,29,13,14,15,1,16]]

# Produtos

## Abt-buy

Leipzig group

In [94]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = 'D:\Dados\OneDrive\Doutorado\Datasets\Leipzig\\'
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'abt-buy'+os.sep

inputZip = 'Abt-Buy.zip'
ds1_name = 'Abt'
ds2_name = 'Buy'
gs_name ='abt_buy_perfectMapping'


zip_file = 'processed_abt-buy.zip'

ds1 = read(ds_files+inputZip,ds1_name)
ds2 = read(ds_files+inputZip,ds2_name)
gs = read(ds_files+inputZip,gs_name)

### Estatisticas

In [95]:
# ds1.pop('description')
# ds1.pop('price')
# ds2.pop('description')
# z = ds2.pop('price')
# del z

In [96]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2]).to_csv(ds_files+'stats_abt-buy.csv',sep=';')

Dataset para deduplicação com:

In [97]:
display(sdf1)
display(sdf2)

,name,description,price,ds
metrica,,,,
count,1081,1081,418,Abt
unique,1081,1081,166,Abt
top,Garmin Nuvi 255W GPS Navigation System - 01000...,Peerless 32' - 50' Articulating Flat Panel TV ...,$49.00,Abt
freq,1,1,18,Abt
nan,0,0,0,Abt


,name,description,manufacturer,price,ds
metrica,,,,,
count,1092,651,1086,590,Buy
unique,1079,559,116,526,Buy
top,LG 5.6 cu.ft. Freestanding Electric Range,Black,Sony,$499.00,Buy
freq,5,11,170,5,Buy
nan,0,0,441,6,Buy


### Problemas nos dados


sem valores nulos

In [98]:
#df = substituir_valores_nulos(df)

In [99]:
gs.columns = ['id1','id2']
da, db = verify_gg4cc(gs)

O gabarito não é de um dataset clean-clean

In [100]:
display(da.head(5))
display(db.head(5))

,id1,id2
40,32625,201935116
41,32625,201935171
61,22042,202145734
62,22042,202238311
66,23006,202456880


,id1,id2
82,23651,202784323
83,37360,202784323
510,32628,207554111
511,32630,207554111
601,34815,208033806


### Solução do problema

In [101]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

Removendo as entidades do DS

In [102]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

Limpando o gabarito

In [103]:
gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

In [104]:
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dados limpos, agora salvando

In [105]:
save_zip(cds1,cds2,gs,estatiscas,out_file + zip_file)

# Citacoes

## DBLP-ACM

In [10]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = 'D:\Dados\OneDrive\Doutorado\Datasets\Leipzig\\'
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'dblp-acm'+os.sep

inputZip = 'DBLP-ACM.zip'
ds1_name = 'DBLP2'
ds2_name = 'ACM'
gs_name ='DBLP-ACM_perfectMapping'


zip_file = 'processed_DBLP-ACM.zip'

ds1 = read(ds_files+inputZip,ds1_name)
ds2 = read(ds_files+inputZip,ds2_name)
gs = read(ds_files+inputZip,gs_name)

### Limpando

In [11]:
# ds1.pop('title')
# z = ds2.pop('title')
# del z
ds1.title = ds1.title.astype(str)
ds2.title = ds2.title.astype(str)
ds1.year = ds1.year.astype(str)
ds2.year = ds2.year.astype(str)
ds1.id = ds1.id.astype(str)
ds2.id = ds2.id.astype(str)

### Estatisticas

In [12]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,id,title,authors,venue,year,ds
metrica,,,,,,
count,2616,2616,2616,2616,2616,DBLP2
unique,2616,2521,2316,5,10,DBLP2
top,journals/sigmod/Danish98,Editor's Notes,Richard T. Snodgrass,VLDB,2003,DBLP2
freq,1,30,25,877,346,DBLP2
nan,0,0,0,0,0,DBLP2


,id,title,authors,venue,year,ds
metrica,,,,,,
count,2294,2294,2280,2294,2294,ACM
unique,2294,2230,2008,5,10,ACM
top,277632,Book reviews,Karl Aberer,International Conference on Management of Data,2001,ACM
freq,1,8,16,797,282,ACM
nan,0,0,14,0,0,ACM


### Limpando os dados

Sem valores nulos

In [13]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2


,id1,id2


Dataset clean-clean

### Salvando os dados

In [14]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)

## DBLP1-Scholar

In [53]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep
ds_files = 'D:\Dados\OneDrive\Doutorado\Datasets\Leipzig\\'
out_file = base_dir + 'workspace'+os.sep+'tl_pprl2'+os.sep+'datasets'+os.sep+'dblp-acm'+os.sep

inputZip = 'DBLP-Scholar.zip'
ds1_name = 'DBLP1'
ds2_name = 'Scholar'
gs_name ='DBLP-Scholar_perfectMapping'


zip_file = 'processed_DBLP-Scholar.zip'

ds1 = read(ds_files+inputZip,ds1_name)
ds2 = read(ds_files+inputZip,ds2_name)
gs = read(ds_files+inputZip,gs_name)

### Limpando

In [54]:
# ds1.pop('title')
# z = ds2.pop('title')
# del z
ds2.columns = ['id','title','authors','venue','year']
ds1.title = ds1.title.astype(str)
ds2.title = ds2.title.astype(str)
ds1.year = ds1.year.astype(str)

#nan em coluna de interiro
ds2.year = ds2.year.fillna(-1)
ds2.year = ds2.year.astype(int)
ds2.year = ds2.year.astype(str)
ds2.year = ds2.year.replace('-1', np.nan)

ds1.id = ds1.id.astype(str)
ds2.id = ds2.id.astype(str)

### Estatisticas

In [55]:
nan_list1 = ds1.isnull().sum()
nan_list2 = ds2.isnull().sum()

estatiscas1 = ds1.describe(include='object')
estatiscas2 = ds2.describe(include='object')

sdf1 = gerar_estatiscas_df(nan_list1,estatiscas1)
sdf1['ds'] = ds1_name

sdf2 = gerar_estatiscas_df(nan_list2,estatiscas2)
sdf2['ds'] = ds2_name

estatiscas = pd.concat([sdf1,sdf2])#.to_csv(ds_files+'stats_abt-buy.csv',sep=';')

display(sdf1)
display(sdf2)

,id,title,authors,venue,year,ds
metrica,,,,,,
count,2616,2616,2398,2405,2616,DBLP1
unique,2616,2521,2084,6,10,DBLP1
top,conf/vldb/ChatziantoniouR97,Editor's Notes,M Franklin,VLDB,2003,DBLP1
freq,1,30,11,861,346,DBLP1
nan,0,0,218,211,0,DBLP1


,id,title,authors,venue,year,ds
metrica,,,,,,
count,64263,64263,64262,49266,29473,Scholar
unique,64263,63188,55883,12580,127,Scholar
top,jlgqeX_SCwIJ,Foreword,ACMS Anthology,"LECTURE NOTES IN COMPUTER SCIENCE,",2000,Scholar
freq,1,50,107,964,2110,Scholar
nan,0,0,1,14997,34790,Scholar


### Limpando os dados

Sem valores nulos

In [56]:
gs.columns = ['id1','id2']
gs.id1 = gs.id1.astype(str)
gs.id2 = gs.id2.astype(str)
da, db = verify_gg4cc(gs)
display(da.head(5))
display(db.head(5))

,id1,id2
1,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,eBnT7lhV2LwJ
2,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,gBVNSFeS4P8J
3,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,VuY9Y49GqXgJ
4,conf/sigmod/AbiteboulBCMM03,AxpQwgyRyLgJ
5,conf/sigmod/AbiteboulBCMM03,Rjb06zlxbLIJ


,id1,id2
2,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,gBVNSFeS4P8J
190,conf/sigmod/BarbaraI94,7N0H3ra1FFsJ
191,conf/sigmod/BarbaraI94,bMUUwjP72hwJ
192,conf/sigmod/BarbaraI94,bTYTn8VG5hIJ
193,conf/sigmod/BarbaraI94,F7azEOfg-Q0J


### Removendo duplicados

In [77]:
remover_da_lista2 = []
remover_da_lista1 = []

for i in da.id1.unique():
    remover_da_lista2.append(da[da.id1 == i].id2.values[-1])

for i in db.id2.unique():
    remover_da_lista1.append(db[db.id2 == i].id1.values[-1])

Removendo as entidades do DS

In [78]:
cds1 = ds1[~ds1.id.isin(remover_da_lista1)]
cds2 = ds2[~ds2.id.isin(remover_da_lista2)]

Limpando o gabarito

In [79]:
gs = gs[~gs.id1.isin(remover_da_lista1)]
gs = gs[~gs.id2.isin(remover_da_lista2)]

In [82]:
da, db = verify_gg4cc(gs)
display(da.head(150))
display(db.head(5))

,id1,id2
11,conf/sigmod/AcharyaAFZ95,_-yXA5HLnqoJ
12,conf/sigmod/AcharyaAFZ95,0lAHCM-lcjIJ
13,conf/sigmod/AcharyaAFZ95,2fO9nM2FoGYJ
14,conf/sigmod/AcharyaAFZ95,cHxtmO9talkJ
52,conf/sigmod/AdaliCPS96,_CkpzhLiR30J
...,...,...
1714,conf/sigmod/SaltenisJLL00,Klh7nU-G4-IJ
1715,conf/sigmod/SaltenisJLL00,LMm4_4_uNNoJ
1716,conf/sigmod/SaltenisJLL00,odsY0ehPPhIJ
1799,conf/sigmod/SrikantA96,9t4Mce85HT4J


,id1,id2


nao e possivel limpar

### Salvando os dados

In [26]:
save_zip(ds1,ds2,gs,estatiscas,out_file + zip_file)